In [12]:
from linear_models import LinearOneVsAllClassifier
import numpy as np
import noise_functions_multi as noise
import time
import ray
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ray.init()

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:20546 to respond...
Waiting for redis server at 127.0.0.1:24934 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8892/notebooks/ray_ui1818.ipynb?token=a4e57112cce7318102ff366e082b0e5a1678931b7a27a95f



{'local_scheduler_socket_names': ['/tmp/scheduler79723446'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store28008333', manager_name='/tmp/plasma_manager96921296', manager_port=62410)],
 'redis_address': '127.0.0.1:20546',
 'webui_url': 'http://localhost:8892/notebooks/ray_ui1818.ipynb?token=a4e57112cce7318102ff366e082b0e5a1678931b7a27a95f'}

In [3]:
linear_models = [LinearOneVsAllClassifier(10, np.load('linear_models/w_{}.npy'.format(i)), np.load('linear_models/b_{}.npy'.format(i)))
                 for i in range(5)]

In [18]:
np.random.randint(10)

4

In [31]:
type(np.random.randint(10))

int

In [35]:
@ray.remote
def test():
    return np.random.randint(10), ';s'

In [36]:
res = [test.remote() for _ in range(5)]
a = ray.get(res)

';s'

In [4]:
X_exp = np.load('linear_experiments_data/X_exp_multi.npy')
Y_exp = np.load('linear_experiments_data/Y_exp_multi.npy')

In [10]:
s = time.time()
results = [noise.grad_desc_nonconvex.remote([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
res = ray.get(results)
print time.time() - s 

14.9078650475


In [13]:
s = time.time()
results = [noise.grad_desc_nonconvex([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
print time.time() - s 

29.3238580227


In [ ]:
[model.evaluate((X_exp[0] + v).reshape(1,-1), Y_exp[0]) for model in linear_models]